In [1]:
# Import libraries
import torch as t
import sys
import os
from torch import Tensor
import random
from functools import partial
import importlib
import circuitsvis as cv
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from transformers import AutoTokenizer, AutoModelForCausalLM
# Get the current working directory (cwd) instead of __file__ in Jupyter Notebook
current_path = os.getcwd()

# Add the parent directory to Python's path
parent_path = os.path.abspath(os.path.join(current_path, "../app/"))
sys.path.insert(0, parent_path)

# Import your module
import importlib
from global_variables import IMAGE_DIR, DATASET_DIR
from prepare_dataset import SyllogismDataset
import helper_functions as h
import plot_utils as pu
t.set_grad_enabled(False)

# Reload the package
#importlib.reload(pu)
#importlib.reload(h)

/Users/geonhee/miniconda3/envs/mechsyllogism/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Figure 4(a) attention head output patching

In [5]:
gpt_2 = t.tensor([[ 9.3844e-04,  2.1899e-03,  6.1569e-04, -7.6720e-05, -6.7424e-05,
         -1.4569e-03,  2.6206e-04,  1.0554e-03,  2.0762e-03,  3.3822e-03,
         -2.8145e-03, -9.1369e-04, -6.3612e-04, -8.6411e-04,  5.5285e-04,
         -1.3990e-03],
        [-1.1899e-03,  3.4110e-04,  2.4257e-03,  1.3581e-06,  7.0759e-03,
          9.6914e-04, -4.0131e-04, -1.3736e-03,  8.0009e-04, -1.7626e-03,
          3.8330e-05,  3.2005e-03, -1.0648e-03, -2.0426e-03, -1.9533e-03,
          1.7834e-04],
        [ 1.0669e-04,  1.2335e-04, -2.2313e-03,  6.8399e-04, -3.6789e-03,
         -6.2263e-04,  6.1303e-04,  2.4786e-03,  1.8169e-03, -1.1460e-03,
         -1.2620e-03, -1.7568e-04,  4.0651e-04,  9.6920e-04,  6.9884e-03,
         -6.0083e-03],
        [ 2.6436e-03,  1.6490e-03, -1.5727e-03, -7.2061e-03, -2.4644e-03,
         -7.1405e-04,  2.0285e-03,  7.1369e-04,  2.4066e-03, -1.1295e-03,
         -4.8522e-03, -1.5159e-03, -1.1911e-03, -6.1434e-03, -1.1330e-04,
         -1.9514e-03],
        [-8.4742e-04, -7.7432e-03, -2.7531e-04, -2.2480e-03, -6.7723e-03,
         -2.3681e-03,  1.2958e-02,  3.6036e-05,  1.1321e-03, -7.4097e-04,
          6.0446e-03,  8.7802e-04,  4.6071e-03, -9.3453e-03, -2.3266e-03,
         -3.5808e-03],
        [-8.1135e-04,  3.1509e-04,  2.6870e-04, -3.3143e-03,  2.4446e-06,
         -4.4504e-03, -1.0753e-04, -1.4975e-03,  4.5093e-02,  2.5980e-04,
         -2.2509e-04, -8.1967e-03, -1.7619e-03, -8.7969e-03, -1.3139e-03,
         -6.4672e-03],
        [-1.9141e-03,  1.6566e-02, -6.5643e-05, -9.0579e-04,  1.1502e-02,
         -8.0176e-03, -4.9729e-03,  9.6335e-03, -5.8477e-03, -4.1443e-03,
         -9.7231e-03, -9.3212e-03, -4.6305e-03, -3.9962e-04,  9.1267e-03,
          6.9733e-02],
        [ 1.3607e-02,  2.1284e-03,  3.4613e-02, -1.7066e-03, -1.0783e-02,
          3.1313e-03,  7.6815e-03, -6.5270e-03, -3.8205e-03, -5.9006e-03,
         -7.7569e-03,  1.6495e-02,  2.3298e-02,  1.7854e-02,  1.7141e-03,
         -4.4911e-03],
        [ 1.0907e-02,  3.3564e-02,  5.3236e-04, -2.3742e-03, -8.0791e-03,
         -1.6034e-02, -4.5171e-03,  2.4218e-03, -1.3850e-04,  1.0261e-02,
          1.3279e-02, -3.7563e-03,  5.2899e-03,  7.0253e-03, -7.5734e-03,
          2.1110e-02],
        [-6.8421e-03, -1.5259e-03, -3.2085e-03,  3.8445e-02, -2.0622e-03,
          6.3579e-03,  7.2057e-03,  1.6366e-02, -2.1485e-03,  4.7246e-03,
          2.4740e-03,  6.5226e-03,  1.1202e-03, -1.2076e-02,  7.9486e-03,
         -3.1224e-03],
        [ 3.0087e-02,  5.1761e-02,  1.4118e-02,  4.9475e-04,  7.8974e-02,
         -6.3582e-04,  1.9368e-02, -3.0971e-03,  1.6670e-02,  3.1294e-03,
          2.0396e-02, -1.2392e-02, -1.2493e-02, -2.0850e-02,  5.6711e-02,
         -8.9275e-03],
        [-9.9304e-03, -2.3691e-02, -3.1411e-03,  1.4042e-02, -2.5436e-02,
         -9.0701e-03, -3.0700e-02,  5.1749e-03, -1.8873e-03, -1.7915e-02,
          3.5951e-01, -4.9599e-03, -1.5676e-03,  5.8935e-03,  1.3221e-02,
          1.4847e-03],
        [-6.6659e-03, -2.7815e-04,  1.8357e-02,  8.6101e-02, -1.1494e-02,
         -2.3038e-02,  3.7890e-03, -5.2081e-03,  5.0888e-02, -1.6967e-02,
          8.4025e-03, -3.6248e-03,  1.4068e-02,  2.2679e-02, -1.0184e-02,
          7.2873e-02],
        [ 7.8104e-03, -3.4969e-03,  1.5493e-02,  4.2789e-03,  1.2715e-02,
          6.4867e-02, -2.1964e-03, -5.8165e-02, -2.1614e-03, -1.1378e-02,
          6.8756e-03,  6.4321e-04,  3.3768e-02,  9.0661e-03, -1.2933e-03,
         -2.3868e-03],
        [-1.4124e-03, -5.8968e-03, -1.3503e-04,  1.9238e-03, -1.5330e-02,
          1.6328e-02, -3.6495e-03, -2.0934e-03, -8.2677e-04, -1.7785e-02,
         -6.5261e-03,  1.2893e-02,  1.7217e-02, -1.3989e-02,  4.1347e-02,
         -1.4516e-03],
        [-9.2051e-06,  9.4253e-03,  5.0366e-04, -1.4770e-03, -2.3942e-03,
          5.7503e-04,  1.6982e-03, -1.9216e-03,  9.4405e-02, -3.0622e-03,
          1.2704e-02,  2.3867e-03, -1.5239e-03, -1.4220e-03,  1.3110e-01,
         -8.5164e-03],
        [ 8.5614e-03, -3.1521e-03, -1.2822e-03,  1.6403e-04,  1.8936e-03,
         -9.4867e-03,  2.6085e-02,  4.1887e-03,  3.6645e-03,  4.3647e-04,
          7.0261e-04,  1.0834e-03,  1.4083e-03, -1.6006e-03, -8.0477e-03,
          2.8770e-03],
        [ 6.9820e-03,  1.8096e-03, -1.3688e-03,  6.7481e-04, -5.0988e-02,
         -1.1443e-03,  2.8168e-03,  1.3303e-02,  1.1845e-02,  1.5284e-03,
         -4.3986e-03, -9.7313e-03,  8.0032e-02, -3.0248e-03,  5.6154e-04,
         -6.7701e-03],
        [-3.6501e-04, -2.9654e-03, -3.9748e-04, -1.4575e-02, -7.1278e-04,
          1.6152e-03,  1.1368e-03,  5.3526e-04, -1.1689e-02,  7.9324e-04,
          4.4367e-03, -2.8967e-02,  4.3414e-02, -2.1373e-03,  1.5328e-03,
         -7.8651e-04],
        [-5.2809e-03,  4.2324e-01, -3.8053e-03,  1.6899e-03,  5.2684e-04,
         -1.4716e-04,  1.9431e-03,  5.4908e-03,  2.8316e-04, -1.3875e-02,
          4.6358e-05,  1.9578e-03,  9.7185e-03, -4.9229e-02, -4.2160e-04,
         -3.2876e-04],
        [ 1.0166e-03,  1.9904e-03,  2.3626e-03, -7.2636e-04, -5.8856e-04,
          1.3864e-02,  4.7716e-05,  3.8381e-02, -4.0934e-04,  5.9124e-04,
          1.6988e-02,  9.6159e-04, -2.5332e-03,  1.2202e-03,  4.0606e-03,
          1.9957e-03],
        [ 1.0663e-03,  6.9884e-04, -1.0288e-03, -1.3612e-04, -2.3704e-02,
         -2.0258e-02,  2.6941e-03,  6.6660e-02, -7.0336e-04, -1.0565e-03,
          1.2057e-02,  3.7261e-02, -1.2185e-03, -2.8388e-04, -6.2776e-04,
          6.8885e-04],
        [-3.2846e-03, -1.3941e-04,  1.4823e-03,  4.0847e-04,  2.6755e-04,
         -1.6001e-02,  4.7852e-04,  2.4252e-02, -3.7563e-04,  1.0802e-03,
          5.9489e-04,  1.8105e-04,  5.0514e-04,  4.8877e-03, -7.5887e-02,
          2.2636e-03],
        [ 2.0704e-03,  1.8522e-03,  7.9677e-06,  2.1646e-02,  1.7811e-03,
         -1.2486e-03,  1.6959e-03, -2.9583e-02,  8.8493e-03,  2.5874e-03,
         -2.6783e-01,  5.0266e-03, -1.1401e-03,  2.6122e-02, -4.6702e-04,
          4.2006e-04]], device='mps:0')

pythia = t.tensor([[ 7.7165e-03, -1.2248e-02,  5.4667e-04, -6.3452e-03, -4.1114e-03,
         -5.4452e-04,  2.8715e-03,  1.3022e-03],
        [ 1.5758e-03, -6.4278e-03, -2.5667e-03,  6.5130e-03,  6.0119e-03,
         -5.0533e-03,  1.2973e-02,  4.1717e-02],
        [-4.2899e-03,  1.0372e-03, -5.1217e-03,  2.5186e-02, -9.5979e-03,
         -1.6507e-03,  5.0302e-03,  1.6200e-02],
        [ 1.5336e-02, -1.4759e-02, -7.6161e-03, -8.7699e-03, -5.3659e-03,
          9.3014e-04, -5.8964e-02, -2.2318e-03],
        [-1.0491e-02,  1.8009e-01,  1.8049e-02,  7.4922e-03,  1.0069e-01,
          6.8919e-03,  2.0931e-03, -8.9138e-03],
        [-4.1508e-03,  6.5224e-02,  1.0688e-02,  1.2276e-02, -1.0596e-02,
          1.1352e-02, -2.0779e-03,  1.2206e-01],
        [-1.9429e-03,  6.9213e-03,  8.7083e-03,  2.4718e-01, -2.8873e-03,
          6.3679e-02, -2.7114e-03, -5.3465e-03],
        [ 7.1429e-02, -1.3619e-02,  9.4242e-02,  1.2109e-01, -1.9007e-02,
          2.1919e-02, -8.6666e-03, -3.7611e-03],
        [ 2.9707e-02,  5.7484e-02, -2.6239e-02,  1.2532e-02, -1.1294e-02,
          5.7836e-02,  8.4268e-03,  3.8058e-01],
        [-1.7449e-03,  2.4180e-02,  1.2542e-01, -1.4713e-04,  4.9454e-02,
         -5.2439e-03,  1.6519e-02,  2.2027e-02],
        [-5.8385e-03, -3.2770e-02, -1.9396e-01,  5.2117e-02,  2.5672e-02,
          2.6126e-02,  8.9366e-02,  4.5363e-04],
        [ 2.4364e-03,  3.4657e-03, -1.3452e-03, -4.6354e-02,  1.2987e-02,
         -9.8807e-03, -7.9781e-03, -7.1503e-02],
        [-1.5915e-01,  2.5761e-01, -6.7068e-03,  8.7811e-06,  1.8503e-01,
         -5.1640e-02,  4.1196e-05,  1.5352e-03],
        [-1.5287e-04,  4.4111e-02, -1.8815e-02,  4.1745e-03,  1.3192e-02,
          1.1796e-01,  8.3864e-04, -8.5559e-04],
        [-4.5995e-03,  9.7878e-03,  3.7057e-01,  7.8002e-06,  6.4382e-04,
         -1.0322e-05, -1.6686e-02, -4.5965e-04],
        [ 2.8886e-02, -3.6268e-03, -3.5801e-03,  7.3645e-03, -5.9785e-02,
          3.3901e-03,  4.2310e-03,  5.4336e-02]], device='mps:0')


print(gpt_2.shape, pythia.shape)

torch.Size([24, 16]) torch.Size([16, 8])
